# Gene's Super-Simple Visualization Examples


## Step 0: Load Required Python Libraries

In [147]:
try:
    import os
    import pandas as pd
    
    os.environ['USE_PYGEOS'] = '0'
    import geopandas

    import holoviews as hv
    import hvplot.pandas
    import panel as pn
    from geoviews import tile_sources as gvts
except ImportError:
    print("A required library could not be found. ")
    raise

## Example 1: Simple Curve

The goal of this plot will be a guided-discovery of the effect of 
$\Delta x$ on estimates of the first derivative of a curve. 

## Define an arbitrary function
We're going to use this as our $f(t)$:

$$
f(t) = \cos{t \over 8} \cdot \cos{3t \over 2}
$$

For $-5\pi \le t \le 5 \pi$ . 


In [157]:
## Will do a 'numerical' approximation of the derivative of a function
import numpy as np

n = 150  # number of sample points in the domain
L = np.pi*10   # domain / input range over which to plot
dt = L/n # space between points

t = np.arange(-L/2, L/2, dx) 
f_t = np.cos(t/8) * np.cos(t/1.5) 


Examine t and f(t). 

In [158]:
t[0:4]

array([-15.70796327, -15.50796327, -15.30796327, -15.10796327])

In [159]:
f_t[0:4]

array([0.19134172, 0.21952442, 0.2387631 , 0.24921077])

`zip` them together.

In [160]:
list( zip(t, f_t)  )[0:4]


[(-15.707963267948966, 0.19134171618254478),
 (-15.507963267948966, 0.2195244165473699),
 (-15.307963267948967, 0.23876309933481074),
 (-15.107963267948968, 0.24921076723743757)]

In [164]:
hv.Curve( 
        zip(t, f_t),
        label="f(t)"
    ).opts(height=300, width=600, color='black') 
# hv.Bars(
#         zip(t, f_t)
#     ).opts(height=300, width=600, xrotation=90, bar_width=1.005, color='yellow') 

:Curve   [x]   (y)

In [165]:
true_df = (np.cos(t/8) * -(np.sin(t/1.5)/1.5)) + (np.cos(t/1.5) * -np.sin(t/8)/8) 


In [166]:
disp = \
    hv.Curve(zip(t, f_t), label="f(t)").opts(color='black') * \
    hv.Curve(zip(t, true_df), label="f '(t)").opts(color='gray', line_dash=(4,4))

disp.opts(
    hv.opts.Curve( height=300, width=600)
)

:Overlay
   .Curve.F_left_parenthesis_t_right_parenthesis            :Curve   [x]   (y)
   .Curve.F_apostrophe_left_parenthesis_t_right_parenthesis :Curve   [x]   (y)

In [146]:
dt_select = pn.widgets.FloatSlider(
    name='dt', 
    start=0.2, end=2.0,
    step=0.1,
    value=0.2
)

@pn.depends(_dx=dt_select)
def plot(_dx):
    ## estimated
    x = np.arange(-L/2, L/2, _dx)
    f_prime = np.cos(x/8) * np.cos(x/1.5)
    estimated_df = np.convolve(
        np.pad(f_prime, 1, mode='edge'), 
        np.array( [1, 0, -1] ) / (2 * _dx), 
        mode='same')[1:-1]
    disp = \
        hv.Curve(zip(t, true_df), "t", "f '(t)", label="TRUE f '(t)").opts(color='gray', line_dash=(4, 4)) *\
        hv.Curve(zip(x, estimated_df), label="EST f '(t)").opts(color='magenta') * \
        hv.Text(-10, 0.6, f"dt = {_dx:.4f}")
    return disp.opts(
        hv.opts.Curve( height=300, width=600, )
    )

disp = pn.Column(
    dt_select, 
    plot)

disp.servable('Effects of dx on Finite Difference Derivative')

Column
    [0] FloatSlider(end=2.0, name='dt', start=0.2, value=0.2)
    [1] ParamFunction(function, _pane=HoloViews)